In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = 'https://ja.wikisource.org/wiki/%E6%BA%90%E6%B0%8F%E7%89%A9%E8%AA%9E_(%E6%B8%8B%E8%B0%B7%E6%A0%84%E4%B8%80%E6%A0%A1%E8%A8%82)'

In [2]:
base_soup = BeautifulSoup(requests.get(base_url).text)
ul = base_soup.find_all('ul')[1]

chapter_dict = {}
for li in ul.find_all('li'):
    chapter_dict[li.text] = f'https://ja.wikisource.org/{li.a["href"]}'


tog = pd.DataFrame.from_dict(chapter_dict, orient='index').reset_index().rename(columns={'index':'chapter',0:'link'})

In [3]:
def get_text(link):
    text = ''
    soup = BeautifulSoup(requests.get(link).text)
    for p in soup.find_all('p')[3:]:
        text += p.text.strip()
    return text

In [4]:
from tqdm import tqdm
tqdm.pandas()

tog['text'] = tog.link.progress_apply(get_text)

100%|██████████| 54/54 [00:15<00:00,  3.49it/s]


In [18]:
from konoha import SentenceTokenizer
tokenizer = SentenceTokenizer()

tog['sents'] = tog['text'].apply(tokenizer.tokenize)
tog_sents = tog[['chapter','sents']]
tog_sents = tog_sents.explode('sents').reset_index(drop=True)
tog_sents

,chapter,sents
0,桐壺,いづれの御時にか、女御、更衣あまたさぶらひたまひけるなかに、いとやむごとなき際にはあらぬが、...
1,桐壺,はじめより我はと思ひ上がりたまへる御方がた、めざましきものにおとしめ嫉みたまふ。
2,桐壺,同じほど、それより下臈の更衣たちは、ましてやすからず。
3,桐壺,朝夕の宮仕へにつけても、人の心をのみ動かし、恨みを負ふ積もりにやありけむ、いと篤しくなりゆき...
4,桐壺,上達部、上人なども、あいなく目を側めつつ、「いとまばゆき人の御おぼえなり。唐土にも、かかる事...
...,...,...
11055,夢浮橋,「いかが聞こえむ」など責められて、「心地のかき乱るやうにしはべるほど、ためらひて、今聞こえむ...
11056,夢浮橋,主人ぞ、この君に物語すこし聞こえて、「もののけにやおはすらむ。例のさまに見えたまふ折なく、悩...
11057,夢浮橋,所につけてをかしき饗応などしたれど、幼き心地は、そこはかとなくあわてたる心地して、「わざと奉...
11058,夢浮橋,人知れずゆかしき御ありさまをも、え見ずなりぬるを、おぼつかなく口惜しくて、心ゆかずながら参りぬ。


In [8]:
import spacy
nlp = spacy.load('ja_core_news_sm')

In [9]:
from spacy.tokens import DocBin

def spacyify(nlp, df,col_name,f_name='spacy_model_output'):
    texts = df[col_name].to_list()
    doc_bin = DocBin()
    for doc in tqdm(nlp.pipe(texts), total=len(texts)):
        doc_bin.add(doc)
    bytes_data = doc_bin.to_bytes()

    f = open(f'serialized_data/{f_name}','wb')
    f.write(bytes_data)
    f.close()

In [19]:
spacyify(nlp, tog_sents, 'sents')

100%|██████████| 11060/11060 [01:01<00:00, 181.11it/s]


In [20]:
tog_sents.to_csv('tog_sents.csv')